In [1]:
import os
import sys
if sys.path[-1] != "../":
    sys.path.append("../")
    os.chdir("../")

os.environ['https_proxy'] = "http://127.0.0.1:15777"
os.environ['http_proxy'] = "http://127.0.0.1:15777"

import numpy as np
import pandas as pd
from IPython.display import display
from random import sample
from transformers import AutoModel, AutoTokenizer

import torch
from utils.util import *
from utils.index import *
from utils.data import *

from hydra import initialize, compose

config = Config()
with initialize(version_base=None, config_path="../data/config/"):
    overrides = [
        "base=NQ320k",
        # "base=MS300k",
        # "++plm=t5",
    ]
    hydra_config = compose(config_name="_example", overrides=overrides)
    config._from_hydra(hydra_config)

loaders = prepare_data(config)

loader_text = loaders["text"]
loader_query = loaders["query"]
text_dataset = loader_text.dataset
query_dataset = loader_query.dataset

# tokenizer = AutoTokenizer.from_pretrained(os.path.join(config.plm_root, config.plm_tokenizer))

[2024-04-03 07:43:15,463] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


[2024-04-03 07:43:16,245] INFO (Config) setting seed to 42...
[2024-04-03 07:43:16,250] INFO (Config) setting PLM to t5...


downloading t5-base


/share/peitian/Envs/adon/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
[2024-04-03 07:43:26,151] INFO (Config) Config: {'adam_beta1': 0.9, 'adam_beta2': 0.999, 'adam_epsilon': 1e-08, 'batch_size': 2, 'bf16': False, 'cache_root': 'data/cache/NQ320k', 'data_format': 'memmap', 'data_root': '/data/TSGen'

In [12]:
# load terms
code_type = "term"
code_tokenizer = "t5"
# for NQ320k
code_length = 26

tokenizer = AutoTokenizer.from_pretrained(os.path.join(config.plm_root, code_tokenizer))

text_codes = np.memmap(
    f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}/codes.mmp",
    mode="r",
    dtype=np.int32
).reshape(len(text_dataset), -1).copy()

In [13]:
indices = range(10)
text_code = text_codes[indices]
text_code[text_code == -1] = 0
display(tokenizer.batch_decode(text_code))
display(tokenizer.batch_decode(np.array(text_dataset[indices]["text"]["input_ids"])[:, :100]))

['<pad> email, mail, marketing, sent, messages, customer, sending, hide, purpose, opt, online, merchant,</s>',
 '<pad> mother, ted, umbrella, met, meet, tracy, mcconnell, mom,</s><pad>',
 '<pad> sperm, fertilization, spermatozoon, vitro, spermatozoa, egg,</s>',
 '<pad> quarterback, nfl, wins, career, brady, football, peyton, manning,</s>',
 '<pad> roanoke, colony, lost, disappeared, dare, raleigh, established,</s>',
 '<pad> africa, african, regions, five, subregions, west, north, six, south, continent,</s><pad>',
 '<pad> mantis, guardians, actress, french, galaxy, spring, infinity, hacker, sleepless,</s>',
 '<pad> frosty, december, hat, christmas, melt, snowman, life, 1969, special,</s>',
 '<pad> acadians, acadia, french, colonial, acadiens,</s><pad>',
 '<pad> banks, outer, graveyard, carolina, roanoke, islands, wright,</s>']

['Email marketing Email marketing is the act of sending a commercial message, typically to a group of people, using email. In its broadest sense, every email sent to a potential or current customer could be considered email marketing. It usually involves using email to send advertisements, request business, or solicit sales or donations, and is meant to build loyalty, trust, or brand awareness. Marketing emails can be sent to a purchased lead list',
 "The Mother ( How I Met Your Mother ) Tracy McConnell, better known as <unk> The Mother '', is the title character from the CBS television sitcom How I Met Your Mother. The show, narrated by Future Ted, tells the story of how Ted Mosby met The Mother. Tracy McConnell appears in 8 episodes from <unk> Lucky Penny '' to <unk> The Time Travelers '' as",
 'Human fertilization Human fertilization is the union of a human egg and sperm, usually occurring in the ampulla of the fallopian tube. The result of this union is the production of a zygote c

In [14]:
# trie = TrieIndex(save_dir=f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}")
# trie.load()

# wordset = WordSetIndex(save_dir=f"data/cache/{config.dataset}/codes/{code_type}/{code_tokenizer}/{code_length}", sep_token_id=6)
# wordset.fit(None)

# text_codes = np.sort(text_codes, axis=-1)
df = pd.DataFrame(text_codes)
duplicates = df.groupby(df.columns.tolist(),as_index=False).size()
duplicates = duplicates.sort_values("size", ascending=False)
duplicates.reset_index(drop=True, inplace=True)

dup = df.duplicated(keep="first").to_numpy()
dup_indices = np.argwhere(dup)[:, 0]
len(dup_indices), duplicates["size"][duplicates["size"] > 1].sum()

(4, 8)